# Training Your Own "Dense Passage Retrieval" Model

EXECUTABLE VERSION: [colab](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial9_DPR_training.ipynb)

Haystack contains all the tools needed to train your own Dense Passage Retrieval model.
This tutorial will guide you through the steps required to create a retriever that is specifically tailored to your domain.


In [2]:
# Here are some imports that we'll need

from haystack.retriever.dense import DensePassageRetriever
from haystack.preprocessor.utils import fetch_archive_from_http

01/06/2021 16:32:35 - INFO - faiss -   Loading faiss with AVX2 support.
01/06/2021 16:32:35 - INFO - faiss -   Loading faiss.


## Training Data

DPR training performed using Information Retrieval data.
More specifically, you want to feed in pairs of queries and relevant documents.

To train a model, we will need a dataset that has the same format as the original DPR training data.
Each data point in the dataset should have the following dictionary structure.

``` python
    {
        "dataset": str,
        "question": str,
        "answers": list of str
        "positive_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
        "negative_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
        "hard_negative_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
    }
```

## Using Question Answering Data

Question Answering datasets can sometimes be used as training data.
Google's Natural Questions dataset, is sufficiently large
and contains enough unique passages, that it can be converted into a DPR training set.
This is done simply by considering answer containing passages as relevant documents to the query.

The SQuAD dataset, however, is not as suited to this use case since its question and answer pairs
are created on only a very small slice of wikipedia documents.

## Download Original DPR Training Data

WARNING: These files are large! The train set is 7.4GB and the dev set is 800MB

We can download the original DPR training data with the following cell.
Note that this data is probably only useful if you are trying to train from scratch.

In [ ]:
# Download original DPR data
# WARNING: the train set is 7.4GB and the dev set is 800MB

doc_dir = "data/dpr_training"

s3_url_train = "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-train.json.gz"
s3_url_dev = "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz"

fetch_archive_from_http(s3_url_train, output_dir=doc_dir)
fetch_archive_from_http(s3_url_dev, output_dir=doc_dir)

## Option 1: Training DPR from Scratch

The default variables that we provide below are chosen to train a DPR model from scratch.
Here, both passage and query embedding models are initialized using BERT base
and the model is trained using Google's Natural Questions dataset (in a format specialised for DPR).

If you are working in a language other than English,
you will want to initialize the passage and query embedding models with a language model that supports your language
and also provide a dataset in your language.

In [4]:
# Here are the variables to specify our training data, the models that we use to initialize DPR
# and the directory where we'll be saving the model

doc_dir = "data/dpr_training"

train_filename = "biencoder-nq-train.json.gz"
dev_filename = "biencoder-nq-dev.json.gz"

query_model = "bert-base-uncased"
passage_model = "bert-base-uncased"

save_dir = "..saved_models/dpr"

## Option 2: Finetuning DPR

If you have your own domain specific question answering or information retrieval dataset,
you might instead be interested in finetuning a pretrained DPR model.
In this case, you would initialize both query and passage models using the original pretrained model.
You will want to load something like this set of variables instead of the ones above

In [ ]:
# Here are the variables you might want to use instead of the set above
# in order to perform pretraining

doc_dir = "PATH_TO_YOUR_DATA_DIR"
train_filename = "TRAIN_FILENAME"
dev_filename = "DEV_FILENAME"

query_model = "facebook/dpr-question_encoder-single-nq-base"
passage_model = "facebook/dpr-ctx_encoder-single-nq-base"

save_dir = "..saved_models/dpr"

## Initialization

Here we want to initialize our model either with plain language model weights for training from scratch
or else with pretrained DPR weights for finetuning.
We follow the [original DPR parameters](https://github.com/facebookresearch/DPR#best-hyperparameter-settings)
for their max passage length but set max query length to 64 since queries are very rarely longer.

In [6]:
## Initialize DPR model

retriever = DensePassageRetriever(
    document_store=None,
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=256
)

01/06/2021 16:33:25 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
01/06/2021 16:33:33 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


## Training

Let's start training and save our trained model!

How long does it take? e.g. on V100?
Batch size / epochs / expected behaviors
What is embed title, its in both the DPR constructor and DPR.train()
What are decent loss values

In [ ]:
# Start training our model and save it when it is finished

retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=4,
    save_dir=save_dir,
    embed_title=True
)

## Loading

Loading our newly trained model is simple!

In [ ]:
reloaded_retriever = DensePassageRetriever.load(save_dir)